# Algoritmos Genéticos

## Aplicación: Problema de las 8 reinas

Usando un tablero regular de ajedrez $8 \times 8$, el problema consiste en colocar $8$ reinas en el tablero de manera que ninguna ataque a las demás.

### Representación

In [ ]:
import numpy
import random
from numpy.random import seed
from numpy.random import randint

Dado el tablero regular $8 \times 8$, en una cadena de 8 dígitos se representan los estados de las 8 reinas. 
Por ejemplo, si guardamos estas posiciones en un arreglo $pos$.
> $pos[i]$: Posición (columna) de la reina $i$, ubicada en la fila $i$.

Los algoritmos genéticos, análogos al proceso de evolución y selección natural, necesitan una **población inicial de estados**. En este caso, inicializamos 4 tableros.

In [ ]:
#Población Inicial
pinicial = ([2,4,7,4,8,5,5,2],[3,2,7,5,2,4,1,1],[2,4,4,1,5,1,2,4],[3,2,5,4,3,2,1,3])
N = 8

### Función de Idoneidad

En un algoritmo genético, por cada iteración, se busca elegir $n$ individuos de la población actual. Entonces, la función de idoneidad mide la capacidad de cada individuo para elegir a los 'más aptos'.

Definimos $Idoneidad$:
> **Entrada:**
> * $lis$: La lista 'individuo', un arreglo de $t$ elementos (regularmente $8$), que representa la posición de las $t$ reinas en el tablero $t \times t$.

La función determina cuántos pares de reinas obedecen las condiciones, es decir, no se atacan entre ellas. 

Dado que se asume una entrada de $8$ reinas, el resultado **óptimo** es: 
> $C_{2}^8 = \frac{8!}{2!(6!)} = \frac{8 \times 7}{2} = \frac{56}{2} = 28$
>
> Es decir, todos los pares posibles del conjunto de 8 reinas posicionadas en el tablero.

Internamente la función inicializa:
* $valores$: arreglo de $t$ elementos, análogo a las reinas. Representa el número de reinas que no son amenazadas por la pieza.
* $suma$: número de pares de reinas que no son amenazadas en todo el tablero.

Ambas empiezan con los valores óptimos. 
- Así, $valores[i] = t-1$; es decir, todas las reinas excepto la reina $i$ están a salvo de esta.
- $suma = 56$, por el análisis combinatorio previo de pares.

Se analiza cada reina contra las demás, bajando el valor de $valores$ y $suma$ por cada vez que las reinas se amenazan.

> **Retorno:**
> * Arreglo $valores$
> * $\frac{suma}{2}$, el resultado de la función idónea.

In [ ]:
#Función de Idoneidad
def Idoneidad(lis):
  t = len(lis)
  valores = [t-1 for i in range(t)]
  suma = 7*t
  for i in range(0,t):
    for j in range(0,t):
      if i < j:
        if lis[i] == lis[j] or lis[i] - (j-i) == lis[j] or lis[i] + (j-i) == lis[j]:
          #Si (Misma columna?) or (Diagonal a la izquierda) or (Diagonal a la derecha)
          valores[i] -= 1
          suma -= 1
      if i > j:
        if lis[i] == lis[j] or lis[i] - (i-j) == lis[j] or lis[i] + (i-j) == lis[j]:
          valores[i] -= 1
          suma -= 1 
  return valores,int(suma/2)

In [ ]:
fido = [None for i in range(len(pinicial))]
for i in range(len(pinicial)):
  vals, fido[i] = Idoneidad(pinicial[i])
  print(vals)
print("\nFunción Idoneidad: ", fido)

[6, 6, 6, 6, 7, 6, 6, 5]
[6, 5, 6, 6, 6, 6, 6, 5]
[5, 5, 3, 6, 7, 4, 5, 5]
[3, 4, 2, 2, 1, 2, 3, 5]

Función Idoneidad:  [24, 23, 20, 11]


### Selección:

Debemos de garantizar que los mejores individuos
tienen una mayor posibilidad de ser padres
(reproducirse) frente a los individuos menos
buenos.
Debemos de ser cuidadosos para dar una
oportunidad de reproducirse a los individuos menos
buenos. Éstos pueden incluir material genético útil
en el proceso de reproducción.
Esta idea nos define la presión selectiva que
determina en qué grado la reproducción está
dirigida por los mejores individuos.

Para cada padre a seleccionar:

*   Escoger aleatoriamente k individuos, con reemplazamiento
*   Seleccionar el mejor de ellos


k se denomina tamaño del torneo. A mayor k, mayor presión
selectiva y viceversa

In [ ]:
#Probabilidad de selección
pselec = [None for i in range(len(pinicial))]
for i in range(len(fido)):
  pselec[i] = (round(100*fido[i]/sum(fido)))
  print(pselec[i])

31
29
26
14


La combinacion de parejas se realiza mediante operadores de cruce

Podríamos tener uno o más operadores de cruce para
nuestra representación.
Algunos aspectos importantes a tener en cuenta son:

*   Los hijos deberían heredar algunas características de cada
padre. 
*   Se debe diseñar de acuerdo a la representación.
*   Se utiliza con una probabilidad alta de actuación sobre cada
pareja de padres a cruzar, si no actua los
padres son los descendientes del proceso de recombinación
de la pareja.


In [ ]:
#Parejas aleatorias
def Parejas(n):
    ale = random.sample(range(int(n/2),n),int(n/2))
    par = {}
    for i in range(int(n/2)):
        par[i] = ale[i]
        par[ale[i]] = i
    return par

In [ ]:
#PAREJAS + CRUCE
parejas = Parejas(len(pinicial))
n1 = 0
nfinal = [None for i in range(len(pinicial))]
for i in range(len(parejas)):
  a = parejas[i]
  if i%2==0:
    n1 = randint(1, N-2)
  if i%2==1:
    Hijo1 = []
    Hijo2 = []
    Hijo1.extend(pinicial[parejas[i]][0:n1])
    Hijo1.extend(pinicial[parejas[i-1]][n1:])
    Hijo2.extend(pinicial[parejas[i-1]][0:n1])
    Hijo2.extend(pinicial[parejas[i]][n1:])
    nfinal[i-1]=Hijo1
    nfinal[i]=Hijo2
for i in nfinal:
  print(i)

[3, 2, 4, 1, 5, 1, 2, 4]
[2, 4, 5, 4, 3, 2, 1, 3]
[3, 2, 7, 4, 8, 5, 5, 2]
[2, 4, 7, 5, 2, 4, 1, 1]


Podemos tener uno o más operadores de mutación para
nuestra representación.
Algunos aspectos importantes a tener en cuenta son:

*   Debe permitir alcanzar cualquier parte del espacio de búsqueda.
*   El tamaño de la mutación debe ser controlado.
*   Se aplica con una probabilidad muy baja de actuación sobre cada
descendiente obtenido tras aplicar el operador de cruce
(incluidos los descendientes que coinciden con los padres porque
el operador de cruce no actúa).

In [ ]:
# MUTACIÓN
for i in range(len(nfinal)):
  prob = random.randint(0,1)
  if prob:
    num = random.randint(0,9)
    pos = random.randint(0,N-1)
    nfinal[i][pos] = num

for i in nfinal:
  print(i)

[3, 2, 4, 1, 2, 1, 2, 4]
[2, 4, 5, 8, 3, 2, 1, 3]
[3, 1, 7, 4, 8, 5, 5, 2]
[2, 4, 7, 5, 2, 4, 1, 1]


## Bibliografía:

1. Berry, N. (2012) . *Eight Queens Problem*. DataGenetics. 
Recuperado de: https://www.datagenetics.com/blog/august42012/

2. 
SCI2S. (2013). BIOINFORMÁTICA TEMA 6: ALGORITMOS GENÉTICOS I: CONCEPTOS BÁSICOS. Recuperado de: https://sci2s.ugr.es/sites/default/files/files/Teaching/GraduatesCourses/Bioinformatica/Tema%2006%20-%20AGs%20I.pdf